In [3]:
import keras
import numpy as np
from PIL import Image
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
from misc import load_data1, load_data2, load_data3, load_data4, load_data5, load_data6, load_data7, load_data8
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D

In [4]:
data = load_data1()

K.set_learning_phase(1)

sess = K.get_session()
X = K.tf.placeholder(shape=(None, 375, 500, 3), dtype=K.tf.float32)
Y = K.tf.placeholder(shape=(None, 5), dtype=K.tf.float32)

In [6]:
vgg = VGGFace(include_top=False, input_tensor = X, input_shape=(375, 500, 3))
for layer in vgg.layers:
    layer.trainable = False

Z1 = (Conv2D(filters=1024, kernel_size=3, strides=1, padding='same'))(vgg.get_layer(index=13).output)
Z2 = (BatchNormalization())(Z1)
Z3 = (Activation('relu'))(Z2)
Z4 = (Conv2D(filters=1024, kernel_size=1, strides=1, padding='same'))(Z3)
Z5 = (BatchNormalization())(Z4)
Z6 = (Activation('relu'))(Z5)
Z7 = (MaxPool2D(pool_size=2, strides=2))(Z6)

Z8 = (Conv2D(filters=256, kernel_size=1, strides=1, padding='same'))(Z7)
Z9 = (BatchNormalization())(Z8)
Z10 = (Activation('relu'))(Z9)
Z11 = (Conv2D(filters=512, kernel_size=3, strides=1, padding='same'))(Z10)
Z12 = (BatchNormalization())(Z11)
Z13 = (Activation('relu'))(Z12)
Z14 = (MaxPool2D(pool_size=2, strides=2))(Z13)

In [7]:
p1 = Conv2D(filters=5, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=13).output)
p2 = Conv2D(filters=5, kernel_size=3, strides=1, padding='same')(Z7)
p3 = Conv2D(filters=5, kernel_size=3, strides=1, padding='same')(Z14)
p4 = Conv2D(filters=5, kernel_size=1, strides=1, padding='same')(Z14)

In [8]:
def NMS(p1, p2, p3, p4):
    pred1 = K.reshape(p1, (46 * 62, 5))
    pred2 = K.reshape(p2, (23 * 31, 5))
    pred3 = K.reshape(p3, (11 * 15, 5))
    pred4 = K.reshape(p4, (11 * 15, 5))

    Z1 = K.tf.image.non_max_suppression(boxes=K.reshape(pred1[:, 1:], ((46 * 62, 4))),
                                        scores=K.reshape(pred1[:, 0], ((46 * 62,))), max_output_size=10,
                                        iou_threshold=0.55)
    Z2 = K.tf.image.non_max_suppression(boxes=K.reshape(pred2[:, 1:], ((23 * 31, 4))),
                                        scores=K.reshape(pred2[:, 0], ((23 * 31,))), max_output_size=10,
                                        iou_threshold=0.55)
    Z3 = K.tf.image.non_max_suppression(boxes=K.reshape(pred3[:, 1:], ((11* 15, 4))),
                                        scores=K.reshape(pred3[:, 0], ((11 * 15,))), max_output_size=10,
                                        iou_threshold=0.55)
    Z4 = K.tf.image.non_max_suppression(boxes=K.reshape(pred4[:, 1:], ((11 * 15, 4))),
                                        scores=K.reshape(pred4[:, 0], ((11 * 15,))), max_output_size=10,
                                        iou_threshold=0.55)

    A = K.concatenate(tensors=[K.gather(pred1, Z1), K.gather(pred2, Z2), K.gather(pred3, Z3), K.gather(pred4, Z4)],
                      axis=0)

    Z5 = K.tf.image.non_max_suppression(boxes=K.reshape(A[:, 1:], (-1, 4)), scores=K.reshape(A[:, 0], (-1,)),
                                        max_output_size=10, iou_threshold=0.6)
    return K.gather(A, Z5)


In [99]:
def sigmoid(X):
    return 1/(1 + K.exp(-X))

def relu(X):
    return K.maximum(X, 0)

def loss(Y, y_, a=0.6):
    if K.tf.size(Y) != K.tf.size(y_):
        y = K.mean(y_, axis = 1)
        y = K.tf.zeros_like(Y) + y
        y_ = y
    softMax = K.mean(-(Y[:, 0] * K.log(sigmoid(y_[:, 0])) + ((1 - Y[:, 0]) * K.log(sigmoid(1 - y_[:, 0])))))
    mse = K.mean(keras.losses.mean_squared_error(Y[:, 1:], relu(y_[:, 1:])))
    return softMax + a*mse 

In [100]:
y_ = NMS(p1, p2, p3, p4)

Loss = loss(Y, y_)

<class 'tensorflow.python.framework.ops.Tensor'>


In [96]:
opt = K.tf.train.AdagradOptimizer(learning_rate=0.0001)
train_ = opt.minimize(Loss)
sess.run(K.tf.global_variables_initializer())

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [97]:
def train(epochs = 10):
    for e in range(epochs):
        cost = 0
        for i, d in enumerate(data):
            _, L = sess.run([train_, Loss], feed_dict={X: np.array(Image.open("x.jpg")).reshape((1, 375, 500, 3)), Y: d[1].reshape((-1, 5))})
            cost += L
            if i % 100 == 0:
                print("epoch {}, iter {} , loss {}".format(e, i , cost/len(data)))

In [98]:
train(epochs = 12)

InvalidArgumentError: Incompatible shapes: [2,5] vs. [10]
	 [[Node: add_53 = Add[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](zeros_like_7, Mean_40)]]
	 [[Node: gradients_14/concat_16_grad/Squeeze/_2697 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_602_gradients_14/concat_16_grad/Squeeze", tensor_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'add_53', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/chawat/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/chawat/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, 

In [71]:
x = np.array([[2, 2, 3], [1, 2, 3], [1, 2, 3]])

In [49]:
x

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [50]:
y = np.array([1, 2, 3])

In [51]:
y

array([1, 2, 3])

In [52]:
x + y

array([[2, 4, 6],
       [2, 4, 6],
       [2, 4, 6]])

In [73]:
x[x[:, 0] > 1]

array([[2, 2, 3]])

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])